### Isolation Forest 

In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
demand_data = pd.read_csv("../simulated_data/simulated_demand.csv")
demand_data.starttime = pd.to_datetime(demand_data.starttime)
demand_data=demand_data.groupby(["location", "car_type"]).resample("5min", on="starttime")["id"].count().reset_index(name="count")
demand_data['hour'] = demand_data.starttime.dt.hour
demand_data = demand_data[['location','car_type', 'hour','count']]


In [212]:
import random
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

ct = ColumnTransformer([("ohe", OneHotEncoder(),['car_type','location'])], remainder="passthrough")
ct.fit(demand_data)
X_train, X_test = train_test_split(demand_data,test_size=0.2)
# X_test['count'] = X_test['count'] 

clf = IsolationForest(contamination=0.04)
pipeline = Pipeline([("ct", ct), ("model", clf)])

pipeline.fit(X_train)
predictions = pipeline.predict(X_test)
print("len ", len(predictions), "sum", sum(predictions))
signature = infer_signature(X_train, pipeline.predict(X_train))
mlflow.sklearn.log_model(pipeline, "isolation_forest", signature=signature)



len  541 sum 499


c:\Users\janguy\Anaconda3\envs\dlresearch\lib\site-packages\mlflow\models\signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)


ModelInfo(artifact_path='isolation_forest', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.8.5', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.0.2', 'serialization_format': 'cloudpickle'}}, model_uri='runs:/6e98d315dbf142dc895bcd92d44dc5b9/isolation_forest', model_uuid='da981cc1c6354ece9864f73d07d1da2c', run_id='6e98d315dbf142dc895bcd92d44dc5b9', saved_input_example_info=None, signature_dict={'inputs': '[{"name": "location", "type": "string"}, {"name": "car_type", "type": "string"}, {"name": "hour", "type": "long"}, {"name": "count", "type": "long"}]', 'outputs': '[{"type": "tensor", "tensor-spec": {"dtype": "int32", "shape": [-1]}}]'}, utc_time_created='2022-11-09 00:32:00.778942')

In [ ]:
pd.read_csv("mlruns/0/6e98d315dbf142dc895bcd92d44dc5b9/artifacts/isolation_forest/")

az ml model create --name isolation_forest --version 1 --type mlflow_model --path ./mlruns/0/6e98d315dbf142dc895bcd92d44dc5b9/artifacts/isolation_forest/ 

In [17]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "Inputs": {
    "data": demand_data.head().to_dict(orient="records")
  }
}

body = str.encode(json.dumps(data))

url = 'https://anomaly-detection.westus.inference.ml.azure.com/score'
api_key = '' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'deployment1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 424
server: azureml-frontdoor
date: Thu, 10 Nov 2022 16:21:32 GMT
content-type: application/json
content-length: 70
x-ms-run-function-failed: False
x-ms-server-version: azmlinfsrv/0.7.6
x-ms-request-id: 8f9277df-0fb9-4dc3-b62a-9535490ad689
x-request-id: 8f9277df-0fb9-4dc3-b62a-9535490ad689
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 400
azureml-model-deployment: deployment1
connection: close


{"message": "A value is not provided for the 'input_data' parameter."}


'[{"location":"loc_0","car_type":"comfort","hour":16,"count":1},{"location":"loc_0","car_type":"comfort","hour":16,"count":1},{"location":"loc_0","car_type":"comfort","hour":16,"count":3},{"location":"loc_0","car_type":"comfort","hour":16,"count":5},{"location":"loc_0","car_type":"comfort","hour":17,"count":2}]'

In [19]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "Inputs": {
    "data": [
      {
        "location": "loc_0",
        "car_type": "comfort",
        "hour": 15,
        "count": 2
      },
      {
        "location": "loc_0",
        "car_type": "comfort",
        "hour": 16,
        "count": 3
      }
    ]
  }
}
data =  {
  "Inputs": {
    "data": demand_data.head().to_dict()
  }
}

body = str.encode(json.dumps(data))

url = 'https://anomaly-detection.westus.inference.ml.azure.com/score'
api_key = 'x02RC8IWFJ5z7l6g3ya290vuyVxGRIjn' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'deployment2' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"[1, 1, 1, 1, 1]"'


In [227]:
model = mlflow.sklearn.load_model(model_uri="mlruns/0/6e98d315dbf142dc895bcd92d44dc5b9/artifacts/isolation_forest/")
model.predict(demand_data.head())

array([1, 1, 1, 1, 1])

'{"location":{"0":"loc_0","1":"loc_0","2":"loc_0","3":"loc_0","4":"loc_0"},"car_type":{"0":"comfort","1":"comfort","2":"comfort","3":"comfort","4":"comfort"},"hour":{"0":16,"1":16,"2":16,"3":16,"4":17},"count":{"0":1,"1":1,"2":3,"3":5,"4":2}}'